In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [2]:
class Network(nn.Module):
    def __init__(self, p1, p2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 60, bias= True),
            nn.ReLU(),
            nn.Linear(60, 60, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p1),
            nn.Linear(60, 55, bias = True),
            nn.ReLU(),
            nn.Linear(55, 50, bias= True),
            nn.ReLU(),
            nn.Linear(50, 50, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p2),
            nn.Linear(50, 45, bias= True),
            nn.ReLU(),
            nn.Linear(45, 40, bias= True),
            nn.ReLU(),
            nn.Linear(40, 35, bias= True),
            nn.ReLU(),
            nn.Linear(35, 30, bias= True),
            nn.ReLU(),
            nn.Linear(30, 25, bias = True),
            nn.ReLU(),
            nn.Linear(25, 20, bias = True),
            nn.ReLU(),
            nn.Linear(20, 15, bias = True),
            nn.ReLU(),
            nn.Linear(15, 10, bias = True),
            nn.ReLU(),
            nn.Linear(10, 5, bias = True)
        )
        self.classify = nn.Sequential(
            nn.Linear(5, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [3]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [4]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=100, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 100)

In [5]:
model = Network(0.3, 0.6)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=60, bias=True)
    (1): ReLU()
    (2): Linear(in_features=60, out_features=60, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=60, out_features=55, bias=True)
    (6): ReLU()
    (7): Linear(in_features=55, out_features=50, bias=True)
    (8): ReLU()
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.6, inplace=False)
    (12): Linear(in_features=50, out_features=45, bias=True)
    (13): ReLU()
    (14): Linear(in_features=45, out_features=40, bias=True)
    (15): ReLU()
    (16): Linear(in_features=40, out_features=35, bias=True)
    (17): ReLU()
    (18): Linear(in_features=35, out_features=30, bias=True)
    (19): ReLU()
    (20): Linear(in_features=30, out_features=25, bias=True)
    (21): ReLU()
    (22): Linear(in_features=25, out_features=20, bias=True)
    (23): ReLU()
    (24): Linear(in_features=20, out_

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001, weight_decay= 0.00001)

In [7]:
for epoch in range(500):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 3321.24760222435
loss - 3311.4658472537994
loss - 3308.8696380853653
loss - 3308.0276544094086
loss - 3307.732715547085
loss - 3307.6249402165413
loss - 3307.601688683033
loss - 3307.486014008522
loss - 3307.554578900337
loss - 3305.4195892214775
loss - 3300.088252067566
loss - 3296.2837846279144
loss - 3293.774529516697
loss - 3293.0360924601555
loss - 3291.3375826478004
loss - 3290.2111157774925
loss - 3289.625064074993
loss - 3289.033482849598
loss - 3288.1224403977394
loss - 3286.4788682460785
loss - 3286.2285990715027
loss - 3285.6048377752304
loss - 3284.830412507057
loss - 3284.0441341400146
loss - 3284.1068559885025
loss - 3283.1753460764885
loss - 3282.3257467746735
loss - 3282.3370019197464
loss - 3282.4154876470566
loss - 3281.1546808481216
loss - 3280.977334201336
loss - 3279.7261368632317
loss - 3280.2690665125847
loss - 3278.797425210476
loss - 3278.675228834152
loss - 3278.0596626400948
loss - 3277.483148932457
loss - 3277.0376688838005
loss - 3276.555063188076
lo

In [8]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.599297767905127
